In [3]:
"""State Definitions and Pydantic Schemas for Logistics Agent.

This defines the state objects and structured schemas used for
the Logistics Agent scoping workflow, including Logistics state management and output schemas.
"""

import operator
from datetime import date
from typing_extensions import Optional, Annotated, List, Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field


# =======================================
# Step 1: Load logitics fields dynamiclly
# =======================================

with open("../IBL_SCHEMA.json", "r", encoding="utf-8") as iblfields:
    ibl_fields = json.load(iblfields)

logistics_fields = ibl_fields.get("logistics_agent", [])

# Dynamically create Pydantic model for shipment fields
DynamicShipmentFields = create_model(
    "DynamicShipmentFields",
    **{
        field_item["field"]: (
            Optional[field_item["dataType"]],  # default type; could later map dataType
            Field(None, description = field_item.get("description", ""))
        )
        for field_item in logistics_fields
    }
)

In [4]:
DynamicShipmentFields.model_fields

{'AWB/BL': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='The unique Air Waybill or Bill of Lading number for the shipment.'),
 'AWB/BL Date': FieldInfo(annotation=Union[date, NoneType], required=False, default=None, description='The issuance date of the Air Waybill or Bill of Lading (YYYY-MM-DD).'),
 'Forwarder': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='The issuance date of the Air Waybill or Bill of Lading (YYYY-MM-DD).'),
 'Incoterm': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='The issuance date of the Air Waybill or Bill of Lading (YYYY-MM-DD).'),
 'Product Temperature': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='The issuance date of the Air Waybill or Bill of Lading (YYYY-MM-DD).'),
 'Packing': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, description='The issuance date of the Air Waybill

In [43]:
import operator
from typing_extensions import Optional, Annotated, List, Sequence
from datetime import date

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from enum import Enum

class LogisticsSchema(BaseModel):
    """Schema for Logisticis Agent."""
    missing_mandatory_fields: List[str] = Field(
        default_factory=list,
        description = "Fields required by the schema that are missing from the provided data"
    )
    missing_optional_fields: List[str] = Field(
        default_factory=list,
        description = "Optional fields that are missing from the provided data"
    )
    ask_for_optional_fields: bool = Field(
        description = "Specifies whether the user should be prompted for optional fields",
        default = True
    )
    needs_user_confirmation: bool = Field(
        description = "Specifies whether user confirmation is required for the current record",
        default = True
    )
    # Nested logistics fields
    Logistics: DynamicShipmentFields

In [22]:
a = LogisticsSchema

In [27]:
a.model_fields

{'missing_mandatory_fields': FieldInfo(annotation=List[str], required=True, description='Fields required by the schema that are missing from the provided data'),
 'missing_optional_fields': FieldInfo(annotation=List[str], required=True, description='Optional fields that are missing from the provided data'),
 'ask_for_optional_fields': FieldInfo(annotation=bool, required=False, default=True, description='Specifies whether the user should be prompted for optional fields'),
 'needs_user_confirmation': FieldInfo(annotation=bool, required=False, default=True, description='Specifies whether user confirmation is required for the current record'),
 'shimpment': FieldInfo(annotation=DynamicShipmentFields, required=True)}

In [46]:
data = {
    "Logistics": {
        "AWB/BL": "ABC123456",
        "Product_Temperature": None,
        "Shipment_Mode": "Air"
    }
}

schema = LogisticsSchema(**data)

#print(schema.Logistics.)  # "ABC123456"
print(schema.Logistics.Shipment_Mode)  # "Air"
print(schema.dict())

AttributeError: 'DynamicShipmentFields' object has no attribute 'Shipment_Mode'

In [49]:
# ===== IBL FIELDS =====
try:
    with open ("../ibl_schema.json" , "r") as config_file:
        logistics_schema_fields = json.load(config_file).get('logistics_agent', [])
except FileNotFoundError:
    print("Error: logistics_schema.json not found. Please create it.")
    exit()
    
# Process the fields
mandatory_fields = [
    item['field'] for item in logistics_schema_fields if item.get('required') is True
]
optional_fields = [
    item['field'] for item in logistics_schema_fields if item.get('required') is False
]

In [50]:
mandatory_fields

['AWB/BL', 'Shipment Mode']

In [7]:
from src.ibl_data_source import ibl_data_source

ModuleNotFoundError: No module named 'src'

In [17]:
# ===== Import Logistics Fields ("Mandatory","Optional") =====
import_logistics_schema = ibl_data_source("../ibl_schema.json","logistics_agent")

logistics_fields = [
    item['field'] for item in import_logistics_schema
]
mandatory_fields = [
    item['field'] for item in import_logistics_schema if item.get('required') is True
]
optional_fields = [
    item['field'] for item in import_logistics_schema if item.get('required') is False
]

In [18]:
logistics_fields 

['AWB/BL',
 'AWB/BL Date',
 'Forwarder',
 'Incoterm',
 'Product Temperature',
 'Packing',
 'Shipping Temp',
 'Gel Pack Expiry Date',
 'Handover to Clearance',
 'Aggregation',
 'Notified FF Date',
 'Green light - Date',
 'Shipment Mode',
 'Logistic Comment',
 'Remark',
 'ASN Importation Date']

In [19]:
mandatory_fields 

['AWB/BL', 'Shipment Mode']

In [20]:
optional_fields 

['AWB/BL Date',
 'Forwarder',
 'Product Temperature',
 'Packing',
 'Shipping Temp',
 'Gel Pack Expiry Date',
 'Handover to Clearance',
 'Aggregation',
 'Notified FF Date',
 'Green light - Date',
 'Logistic Comment',
 'Remark',
 'ASN Importation Date']

In [21]:
 [ fields for fields in logistics_fields if fields in mandatory_fields ]

['AWB/BL', 'Shipment Mode']